<a href="https://colab.research.google.com/github/mahaksinhal/merit-list-and-scholarship-allocation/blob/main/LEX_and_YACC_Compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEX and YACC Compiler in Colab

Drawbacks:
* Regular interrupts (Ctrl+D, Ctrl+C) for shell won't work in Colab while inputting for program.
<br>Workaround: Store your inputs in a txt file and pass it to the program.

In [ ]:
#@title Install *prerqeuisites* (run this cell first to work on LEX/YACC)
!sudo apt install flex bison

## Lex only

In [1]:
#@title Writing Lex program
%%writefile dag.l
%{
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define MAX 50

// Node structure for DAG
struct Node {
    char op;
    char left[10];
    char right[10];
    int index;
} dag[MAX];

int node_count = 0;

// Function declarations
int search(char op, char *left, char *right);
void insert(char op, char *left, char *right);
void display();

// helper to remove spaces
void trim(char *str);

%}

%option noyywrap

%%
[ \t\n]       ;
[a-zA-Z]      { printf("Identifier: %s\n", yytext); }
"+"|"-"|"*"|"/" {
                    char op = yytext[0];
                    printf("Operator: %c\n", op);
                }
.              ;
%%

// === Helper and DAG functions ===

void trim(char *str) {
    char *p1 = str, *p2 = str;
    while(*p2 != '\0') {
        if(*p2 != ' ')
            *p1++ = *p2;
        p2++;
    }
    *p1 = '\0';
}

int search(char op, char *left, char *right) {
    for (int i = 0; i < node_count; i++) {
        if (dag[i].op == op &&
            strcmp(dag[i].left, left) == 0 &&
            strcmp(dag[i].right, right) == 0)
            return i;
    }
    return -1;
}

void insert(char op, char *left, char *right) {
    int pos = search(op, left, right);
    if (pos == -1) {
        dag[node_count].op = op;
        strcpy(dag[node_count].left, left);
        strcpy(dag[node_count].right, right);
        dag[node_count].index = node_count + 1;
        node_count++;
    }
}

void display() {
    printf("\n--- DAG Representation ---\n");
    printf("Index\tOp\tLeft\tRight\n");
    for (int i = 0; i < node_count; i++) {
        printf("%d\t%c\t%s\t%s\n", dag[i].index, dag[i].op, dag[i].left, dag[i].right);
    }
    printf("--------------------------\n");
}

int main() {
    char expr[100];
    printf("Enter an expression (e.g., a+b*c+b*c): ");
    scanf("%s", expr);

    trim(expr);

    // Simple parser: left-to-right for binary ops (*, +)
    // (For simplicity, assumes valid input and no parentheses)
    for (int i = 0; i < strlen(expr); i++) {
        if (expr[i] == '+' || expr[i] == '-' || expr[i] == '*' || expr[i] == '/') {
            char left[10] = { expr[i - 1], '\0' };
            char right[10] = { expr[i + 1], '\0' };
            insert(expr[i], left, right);
        }
    }

    display();
    yylex();
    return 0;
}


Writing dag.l


if you want to use at txt as an input

In [ ]:
%%writefile program.txt

This is a sample file.

Writing program.txt


In [ ]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

lex -l program.l
gcc lex.yy.c
./a.out program.txt

Word Count: 5
Character Count: 18
Space Count: 4
Line Count: 2


## Lex and Yacc combined

In [ ]:
#@title Writing YACC program
%%writefile program.y

%{
    #include<stdio.h>
    #include<stdlib.h>
%}
%token DIGIT LETTER UND NL
%%
stmt: variable NL {printf("Valid Identifier\n");exit(0);}
variable: LETTER alphanumeric;
alphanumeric: LETTER alphanumeric
            | DIGIT alphanumeric
            | UND alphanumeric
            | LETTER
            | DIGIT
            | UND;
%%

int yyerror(){
    printf("Invalid Identifier\n");
    exit(0);
}

void main(){
    printf("Enter the variable name: ");
    yyparse();
}

Overwriting program.y


In [ ]:
#@title Writing Lex program
%%writefile program.l

%{
    #include "y.tab.h"
%}
%%
[a-zA-Z] {return LETTER;}
[0-9] {return DIGIT;}
[_] {return UND;}
\n {return NL;}
. {return yytext[0];}
%%

Overwriting program.l


if you want to use at txt as an input

In [ ]:
%%writefile program.txt

This is a sample file.

In [ ]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

yacc -d program.y
lex program.l
cc y.tab.c lex.yy.c -ll
./a.out

y.tab.c: In function ‘yyparse’:
y.tab.c:1121:16: warning: implicit declaration of function ‘yylex’ [-Wimplicit-function-declaration]
       yychar = yylex ();
                ^~~~~
y.tab.c:1256:7: warning: implicit declaration of function ‘yyerror’; did you mean ‘yyerrok’? [-Wimplicit-function-declaration]
       yyerror (YY_("syntax error"));
       ^~~~~~~
       yyerrok
Enter the variable name: variable_name
Valid Identifier
